In [322]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
import numpy as np
import time
from random import randint
import socks
import socket
import stem.process

In [66]:
source = []
for i in range(1, 28):
    link = 'https://doulamatch.net/list/birth/tx/' + str(i)
    source.append(link)
    
doula_link_list = []
for link in source:
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    for link in soup.find_all(class_='btn btn-info', href=True):
        doula_link_list.append(link['href'])

tx_doula_link = list(set(doula_link_list))

In [332]:
df = pd.DataFrame(columns = ['Doula ID', 'Website', 'Birth Fee', 'Type of Practice', 'Client per Month', 'Location', 
                             'Experience', 'Certification', 'Review'])

SOCKS_PORT = 9050
tor_process = stem.process.launch_tor_with_config(config = {'SocksPort': str(SOCKS_PORT),},)
socks.setdefaultproxy(proxy_type = socks.PROXY_TYPE_SOCKS5, addr="127.0.0.1", port = SOCKS_PORT)
socket.socket = socks.socksocket

for i in range(len(tx_doula_list)):
    link = 'https://doulamatch.net' + tx_doula_link[i]
    doula_id = re.findall(r'\d+' ,tx_doula_link[i])[0]
    print('Extracting doula number ' + doula_id)
    sleep_time = randint(60, 90)
    print('Sleep for ' + str(sleep_time) + ' secs')
    time.sleep(sleep_time)
    doula_website = ''
    doula_fee = ''
    doula_type_pract = ''
    doula_client_per_month = ''
    doula_location = ''
    doula_exp = ''
    num_cert = ''
    num_review = ''
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    b_tag = soup.find_all('b')
    for tag in b_tag:
        if tag.string == 'Website: ':
            doula_website = tag.next_sibling.string
        elif tag.string == 'Birth Fee:':
            doula_fee = tag.next_sibling.string
        elif tag.string == 'Type of Practice: ':
            doula_type_pract = tag.next_sibling.string
        elif tag.string == 'Clients per Month: ':
            doula_client_per_month = tag.next_sibling.string
        elif tag.string == 'Area of practice:':
            doula_location = tag.next_sibling.string
    strong_tag = soup.find_all('strong')
    for tag in strong_tag:
        if tag.string == 'Birth Doula Experience: ':
            doula_exp = tag.next_sibling.string
    badges = soup.find_all(class_ = 'badge badge-info')
    for badge in badges:
        if 'Certifications' in badge.previous_sibling.string:
            num_cert = badge.string
        elif 'Testimonials' in badge.previous_sibling.string:
            num_review = badge.string        
    df = df.append({'Doula ID': doula_id, 'Website': doula_website, 'Birth Fee': doula_fee, 
                    'Type of Practice': doula_type_pract, 'Client per Month': doula_client_per_month, 
                    'Location': doula_location, 'Experience': doula_exp, 'Certification': num_cert, 
                    'Review': num_review}, ignore_index = True)
    print('Completed processing: index number ' + str(i))
    
tor_process.kill()

Extracting doula number 19073
Sleep for 84 secs
Completed processing: index number 0
Extracting doula number 20901
Sleep for 74 secs
Completed processing: index number 1
Extracting doula number 17409
Sleep for 88 secs
Completed processing: index number 2
Extracting doula number 6615
Sleep for 66 secs
Completed processing: index number 3
Extracting doula number 13374
Sleep for 81 secs
Completed processing: index number 4


In [336]:
def generate_link(num):
    link = 'https://doulamatch.net' + tx_doula_link[num]
    return(link)
generate_link(4)

'https://doulamatch.net/profile/13374/lizet-guajardo'

In [344]:
df.to_csv('./raw_data.csv')

In [343]:
df

,Doula ID,Website,Birth Fee,Type of Practice,Client per Month,Location,Experience,Certification,Review
0,19073,http://www.motherlyriverofbirth.com/,$400 to $600,Solo practice,2 to 4,"Burnet,",1 years and 2 births attended,,
1,20901,,$700 to $900,Solo practice,0 to 4 births and 0 to 4 postpartum families,"San Antonio,",5 years,,
2,17409,https://marissakurrin.com,$800 to $1000,Solo practice,1 to 3,"Carrollton,",10 years and 42 births attended,,2
3,6615,http://www.nobarriersbirth.com,Not specified,Doula agency owner,,"Lindale,",10 years,1,12
4,13374,,$800,Solo practice,1 to 4 births and 1 to 2 postpartum families,"Houston,",3 years and 30 births attended,1,2
5,16943,http://www.kingwooddoula.com,$500 to $2000,Solo practice,1 to 2 births and 1 to 2 postpartum families,"Humble,",1 years and 5 births attended,,5
6,7593,http://www.northdallasdoulas.com,Not specified,Partnership with 2 or more doulas,0 to 6,"Dallas,",6 years and 225 births attended,2,85
7,10566,www.thejourneybirthservices.com,$850 to $1200,Solo practice,1 to 4,"Humble,",4 years and 134 births attended,1,24
8,17652,http://northtexasbirthservices.com,$800 to $950,Solo practice,1 to 4 births and 1 to 4 postpartum families,"Keller,",1 years and 2 births attended,1,1
9,14169,Http://www.katepavlovsky.com,$1000 to $2500,Solo practice,1 to 3,"Austin,",2 years and 30 births attended,1,3


In [342]:
# In HMH
SOCKS_PORT = 9050
tor_process = stem.process.launch_tor_with_config(config = {'SocksPort': str(SOCKS_PORT),},)
socks.setdefaultproxy(proxy_type = socks.PROXY_TYPE_SOCKS5, addr="127.0.0.1", port = SOCKS_PORT)
socket.socket = socks.socksocket

for i in range(5, 50):
    link = 'https://doulamatch.net' + tx_doula_link[i]
    doula_id = re.findall(r'\d+' ,tx_doula_link[i])[0]
    print('Extracting doula number ' + doula_id)
    sleep_time = randint(60, 90)
    print('Sleep for ' + str(sleep_time) + ' secs')
    time.sleep(sleep_time)
    doula_website = ''
    doula_fee = ''
    doula_type_pract = ''
    doula_client_per_month = ''
    doula_location = ''
    doula_exp = ''
    num_cert = ''
    num_review = ''
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    b_tag = soup.find_all('b')
    for tag in b_tag:
        if tag.string == 'Website: ':
            doula_website = tag.next_sibling.string
        elif tag.string == 'Birth Fee:':
            doula_fee = tag.next_sibling.string
        elif tag.string == 'Type of Practice: ':
            doula_type_pract = tag.next_sibling.string
        elif tag.string == 'Clients per Month: ':
            doula_client_per_month = tag.next_sibling.string
        elif tag.string == 'Area of practice:':
            doula_location = tag.next_sibling.string
    strong_tag = soup.find_all('strong')
    for tag in strong_tag:
        if tag.string == 'Birth Doula Experience: ':
            doula_exp = tag.next_sibling.string
    badges = soup.find_all(class_ = 'badge badge-info')
    for badge in badges:
        if 'Certifications' in badge.previous_sibling.string:
            num_cert = badge.string
        elif 'Testimonials' in badge.previous_sibling.string:
            num_review = badge.string        
    df = df.append({'Doula ID': doula_id, 'Website': doula_website, 'Birth Fee': doula_fee, 
                    'Type of Practice': doula_type_pract, 'Client per Month': doula_client_per_month, 
                    'Location': doula_location, 'Experience': doula_exp, 'Certification': num_cert, 
                    'Review': num_review}, ignore_index = True)
    print('Completed processing: index number ' + str(i))
    
tor_process.kill()

Extracting doula number 16943
Sleep for 71 secs
Completed processing: index number 5
Extracting doula number 7593
Sleep for 76 secs
Completed processing: index number 6
Extracting doula number 10566
Sleep for 69 secs
Completed processing: index number 7
Extracting doula number 17652
Sleep for 75 secs
Completed processing: index number 8
Extracting doula number 14169
Sleep for 72 secs
Completed processing: index number 9
Extracting doula number 20456
Sleep for 80 secs
Completed processing: index number 10
Extracting doula number 15927
Sleep for 66 secs
Completed processing: index number 11
Extracting doula number 16639
Sleep for 88 secs
Completed processing: index number 12
Extracting doula number 12605
Sleep for 75 secs
Completed processing: index number 13
Extracting doula number 19747
Sleep for 60 secs
Completed processing: index number 14
Extracting doula number 21042
Sleep for 73 secs
Completed processing: index number 15
Extracting doula number 4252
Sleep for 68 secs
Completed pro

KeyboardInterrupt: 